# 1 start business logic pods on Azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-turtlebot3`.

In [ ]:
export PJ_ROOT="${HOME}/example-turtlebot3"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-turtlebot3
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-turtlebot3`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr8='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr8='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## add auth tokens for the web app of example-turtlebot3

### create new `secrets/auth-tokens.json`

example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "nrWtb8sS0MmwlkhHXv0DC6orPMpFFbni",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
        }
      ],
      "basic_auths": [],
      "no_auths": {}
    }
  },
  {
    "host": "kibana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "yW7FvSGD",
          "password": "6BoTFE5xfUlX3ssV",
          "allowed_paths": ["^.*$"]
        }
      ],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "grafana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": ["^.*$"]
      }
    }
  },
  {
    "host": "web\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "Udgzdg6xMD5ymtQlInFHsM5UVD9OA2Wi",
          "allowed_paths": [
            "^/visualizer/positions/$"
          ]
        }
      ],
      "basic_auths": [
        {
          "username": "1JMF6D46",
          "password": "6u5M0bUhfjj7wMdM",
          "allowed_paths": [
            "/controller/web/",
            "/visualizer/locus/"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": [
          "^.*/static/.*$"
        ]
      }
    }
  }
]
```

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.|=.+[{
  "host": "web\\..+$",
  "settings": {
    "bearer_tokens": [
      {
        "token": "'$(randomstr32)'",
        "allowed_paths": ["^/visualizer/positions/$"]
      }
    ],
    "basic_auths": [
      {
        "username": "'$(randomstr8)'",
        "password": "'$(randomstr16)'",
        "allowed_paths": ["/controller/web/", "/visualizer/locus/"]
      }
    ],
    "no_auths": {
      "allowed_paths": ["^.*/static/.*$"]
    }
  }
}]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json

## change the auth-tokens to kubernetes secrets

### watch `auth` log

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "kubectl logs -f -lapp=auth --all-containers=true"

### delete and re-register auth-tokens to kubernetes secrets

In [ ]:
kubectl delete secret auth-tokens

In [ ]:
kubectl create secret generic auth-tokens --from-file=${CORE_ROOT}/secrets/auth-tokens.json

### confirm the token will be reloaded

**wait a few minutes until the change of secret is detected by Kubernetes.**  
When the new secret is detected, the tokens of auth will be reloaded automatically.

log messages like below will be shown after tokens is reloaded.

```
...
--------
2019/05/21 01:51:07 hosts: [api\..+$ kibana\..+$ grafana\..+$ web\..+$]
--------
2019/05/21 01:51:07 bearerTokenAllowedPaths: map[api\..+$:map[XbZX1LpVv7DG9fu1X3WUq5kiqZyF34zI:[^/orion/.*$ ^/idas/.*$ ^/comet/.*$]] web\..+$:map[Udgzdg6xMD5ymtQlInFHsM5UVD9OA2Wi:[^/visualizer/positions/$]]]
--------
2019/05/21 01:51:07 basicAuthPaths, map[kibana\..+$:map[^.*$:map[1IGQBVF5:zRa2mxZVdBOyO6Zd]] web\..+$:map[/controller/web/:map[1JMF6D46:6u5M0bUhfjj7wMdM] /visualizer/locus/:map[1JMF6D46:6u5M0bUhfjj7wMdM]]]
--------
2019/05/21 01:51:07 noAuthPaths, map[grafana\..+$:[^.*$] web\..+$:[^.*/static/.*$] api\..+$:[] kibana\..+$:[]]
--------
```

## start command proxy service on Azure AKS

In [ ]:
kubectl apply -f controller/cmd-proxy-service.yaml

In [ ]:
envsubst < controller/cmd-proxy-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=cmd-proxy

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
cmd-proxy-d7cbc6dfb-6hnsp   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-v6rrx   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-zf2fd   1/1       Running   0          46s
```

In [ ]:
kubectl get services -l app=cmd-proxy

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
cmd-proxy   ClusterIP   10.0.172.56   <none>        8888/TCP   1m
```

## start robot visualization service on Azure AKS

In [ ]:
kubectl apply -f controller/robot-visualization-service.yaml

In [ ]:
env BEARER_AUTH=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[]|select(.host == "web\\..+$")|.settings.bearer_tokens | map(select(.allowed_paths[] | contains("^/visualizer/positions/$"))) | .[0].token' -r) envsubst < controller/robot-visualization-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=robot-visualization

example)
```
NAME                                   READY     STATUS    RESTARTS   AGE
robot-visualization-7d69bcdff7-g7p98   1/1       Running   0          13m
robot-visualization-7d69bcdff7-k6mb9   1/1       Running   0          13m
robot-visualization-7d69bcdff7-txsqn   1/1       Running   0          13m
```

In [ ]:
kubectl get services -l app=robot-visualization

example)
```
NAME                  TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
robot-visualization   ClusterIP   10.0.112.72   <none>        8888/TCP   13m
```

### register DNS A Record for business logic

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "web" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
curl -i https://web.${DOMAIN}/controller/web/

example)
```
HTTP/1.1 401 Unauthorized
www-authenticate: Basic realm="basic authentication required"
content-length: 0
date: Tue, 18 Jun 2019 05:01:31 GMT
server: envoy
```